# General PowerShell Exploration
Some general things about PowerShell, useful for the everyday, and to be able to leverage more of the capabilities.

Contents:
- [PSDrives](#PSDrives): access all the things in a familiar way; variables, environment, registry, certificates, functions, etc.
    - [Some examples of drives](#Some-examples-of-drivesSome-examples-of-drives)
    - [Some examples of using drives](#Some-examples-of-using-drives)

## PSDrives
In PowerShell, things are accessible as "drives" to it easy to interact with them in a familiar way.  For more information, `Get-Help about_Providers`, or see the online version at Microsoft Docs [About Providers](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_providers). "The data is presented in a consistent format that resembles a file system drive", so we can interact with it in the same way we do a filesystem. We can use drives to explore, troubleshoot, set, unset, etc. the things around us in an easy, direct way.

### Some examples of drives
There are several default PSDrives available in a PowerShell session.
- `Environment`: "environment variables are dynamically named variables that describe the
environment in which your programs run"; `Get-Help about_Environment_Provider`
- `Variables`: the way in which we refer to the objects available in our PowerShell session; `Get-Help about_Variable_Provider`
- `Registry`: you know, the Windows registry, where there are infinite settings available to mess with Windows; `Get-Help about_Registry_Provider`

To get the help topics for the rest of the built-in PowerShell providers, `Get-Help -Category HelpFile -Name *provider*`

### Some examples of using drives

In [3]:
## get the current PSDrives available in this PowerShell session
Get-PSDrive | Select-Object -First 5


Name           Used (GB)     Free (GB) Provider      Root                                CurrentLoc
                                                                                              ation
----           ---------     --------- --------      ----                                ----------
Alias                                  Alias                                                       
C                 433.85         42.53 FileSystem    C:\                                 …ills\docs
Cert                                   Certificate   \                                             
Env                                    Environment                                                 
Function                               Function                                                    



We can `Get-ChildItem` (alias of `dir`) using the "path" of the Variable PSDrive to explore current variables/values

In [5]:
## get some variables
dir variable:\ | Select-Object -First 5


Name                           Value
----                           -----
$                              variable:\
?                              True
^                              dir
args                           {}
ConfirmPreference              High



In [7]:
## get some variables, filtering variable name with wildcards in a familiar way
dir variable:\*preference* | Select-Object -First 5


Name                           Value
----                           -----
ConfirmPreference              High
DebugPreference                SilentlyContinue
ErrorActionPreference          Continue
ProgressPreference             Continue
VerbosePreference              SilentlyContinue



And, we can see what environment variables are set

In [16]:
## have we a environment variable set for a web proxy?
dir env:\http*proxy


Name                           Value
----                           -----
https_proxy                    https://myproxy.dom.com

